In [1]:
using LightGraphs
using GraphIO

function simulateLin(G, init_nodes)
	
	num_edges = ne(G)
	num_vertices = nv(G)
	active = init_nodes

	#generate the tresholds

	tsh = rand(num_vertices)

	#initialize weight array

	weights = ones(num_edges)

	#initialize active flag array

	active = init_nodes

	#degree array

	deg = degree(G)

	#set weights

	for (i, e) in enumerate(edges(G))
		weights[i] = 1/deg[dst(e)]
	end
	
	something_happened = true

	while(something_happened)
		something_happened = false
        
        #we use an array to keep track, for each node, the sum of the weights of all active neighbors
        
		sums = zeros(num_vertices)
		
        #we iterate over all edges and add the sum of the weights of all active neighbors
        
		for (i, e) in enumerate(edges(G))
			if active[src(e)] == 1
				sums[dst(e)] = sums[dst(e)] + weights[i]
			end
		end
        		
        #we then iterate over all nodes and activate the ones where the sum surpasses the treshold
        
		for v in vertices(G)
			if(active[v] == 0 && sums[v] > tsh[v])
				active[v] = 1
                something_happened = true
			end
		end
    end
	return active
	
end

function simulateCasc(G, init_nodes)

	num_edges = ne(G)
	num_vertices = nv(G)
	active = init_nodes

	prob = rand(num_vertices)
	
	#vector that indicates which nodes just became active. 0 = never was active ; 1 = just became active ; 2 = already was active
	
	just_became_active = copy(init_nodes)
	
	something_happened = true
	
	while(something_happened)
		something_happened = false
		
		#iterate over all nodes that just became active
		
		for (v, state) in enumerate(just_became_active)
			if state == 1
				
				#if an active node was considered, set its state to 2 so it doesn't get considered again
				
				just_became_active[v] = 2
				
				#we iterate over all neighbors of a node that just became active
				
				for n in neighbors(G, v)
					
					#if the neighbor is not active
					
					if(active[n] == 0)
						
						#toss a coin wether he becomes active
						
						coin = rand()
						
						if coin > prob[n]
							
							#if it becomes active, set both as active and just became active
							
							active[n] = 1
							just_became_active[n] = 1
							something_happened = true
						end
					end
				end
			end			
		end
	end
	
	return active
	
end

function greedyLin(G, giveaway_number, num_sim)
	num_sim_real = 0
	num_edges = ne(G)
	num_vertices = nv(G)
	
	active = zeros(num_vertices)
	
	for i in range(1, giveaway_number)
		high_score = 0
		high_score_edge = 0
		
		for e in vertices(G)
			if(active[e] == 0)
				init_nodes = copy(active)
				init_nodes[e] = 1
				score = 0
				
				for j in range(1,num_sim)
					score = score + sum(simulateLin(G, init_nodes))
					num_sim_real = num_sim_real +1
				end
				
				if(score > high_score)
					high_score = score
					high_score_edge = e
				end
			end
		end
		
		active[high_score_edge] = 1
		
	end
	
    print("Linear Model. Total number of simulations : ")
	println(num_sim_real)
    print("k = ")
    print(giveaway_number)
    print("; numSim = ")
    println(num_sim)
	print("Number of active nodes after completing the simulation: ")
	print(sum(simulateLin(G, active)))
    print(" out of ")
    println(num_vertices)
    println("")
    
    return active
	
end

function greedyCasc(G, giveaway_number, num_sim)
	num_sim_real = 0
	num_edges = ne(G)
	num_vertices = nv(G)
	
	active = zeros(num_vertices)
	
	for i in range(1, giveaway_number)
		high_score = 0
		high_score_edge = 0
		
		for e in vertices(G)
			if(active[e] == 0)
				init_nodes = copy(active)
				init_nodes[e] = 1
				score = 0
				
				for j in range(1,num_sim)
					score = score + sum(simulateCasc(G, init_nodes))
					num_sim_real = num_sim_real +1
				end
				
				if(score > high_score)
					high_score = score
					high_score_edge = e
				end
			end
		end
		
		active[high_score_edge] = 1
		
	end
	
	print("Cascadel Model. Total number of simulations : ")
	println(num_sim_real)
    print("k = ")
    print(giveaway_number)
    print("; numSim = ")
    println(num_sim)
	print("Number of active nodes after completing the simulation: ")
	print(sum(simulateCasc(G, active)))
    print(" out of ")
    println(num_vertices)
    println("")
    
	return active
	
end
					
	

G = loadgraph("/home/chris/Uni/GraphMining/Programming/Progetto/Graph/3980.edges", "facebook", EdgeListFormat())

resLin = greedyLin(G, 10, 10)
resCasc = greedyCasc(G, 10, 10)

println(sum(resLin))
println(sum(resCasc))

println(sum(simulateLin(G,resLin)))
println(sum(simulateCasc(G,resCasc)))

Linear Model. Total number of simulations : 4750
k = 10; numSim = 10
Number of active nodes after completing the simulation: 17.0 out of 52

Cascadel Model. Total number of simulations : 4750
k = 10; numSim = 10
Number of active nodes after completing the simulation: 47.0 out of 52

17.0
47.0
30.0
51.0
